In [143]:
import glob #filenames and pathnames utility
import os   #operating sytem utility
import numpy as np
import pandas as pd
import pickle
import random as rand
import sys
import math
from BA_C import BA
from scipy.optimize import curve_fit

In [144]:
def linear(df,a,b):
    return (a*df + b)

In [145]:
"""data_directory: folder that contains the csv files for a method and replicate.
fileset: Name of the csv files.
replicate_name: Part of the csv filename that identifies the replicate number.
"""

data_directory = '/Users/sns9/Research/Transcription/SingleCellDistributions_2_CC/Microscopy_HCR_TAMRA_Rep3'
fileset = 'Microscopy_HCR_TAMRA_1818.'
replicate_name = '_Rep3'
current_dir = os.getcwd()
os.chdir(data_directory)

In [146]:
def extract_data(data_set,index_set):
    extracted_data = []

    for i in index_set:
        try:
            extracted_data.append(data_set.values[i])
        except IndexError:
            print(i)
            sys.exit()

    return np.array(extracted_data)

In [147]:
response_files = glob.glob('*.csv')

glob_max = -1
glob_min = 1e12

datas = {}
conclist = []

samples = 1e12

for f in response_files:
    if fileset in f:
        concname = f
        conc = concname.replace(fileset,'')
        conc = int(conc.replace(replicate_name+'.csv',''))
        
        if conc not in conclist:
            temp_d = pd.read_csv(f).to_numpy()[:,0]
            
            datas[int(conc)] = temp_d
            
            glob_max = max(glob_max,max(datas[conc]))
            glob_min = min(glob_min,min(datas[conc]))
            
            conclist.append(conc)
            
conclist.sort()
print(glob_max,glob_min)
conclist

# Offset to shift all the expression values >= 1
offset = -glob_min + 1

bins = []

for conc in conclist:
    datas[conc] += offset
    
    """Get the number of bins for each concentration determined by Freedman-Diaconis rule.
    Choice of linear scale (typically for RNAs) or log scale (typically for Proteins). 
    """
    
    hist, bin_edges = np.histogram(datas[conc],bins='fd')
    
    bins.append(len(hist))
    
print('bins: ',bins)

38774967.0 128952.0
bins:  [560, 55, 55, 98, 90, 57, 86, 101]


In [148]:
"""Freedman-Diaconis ('fd') is only a rule of thumb to estimate the number of bins. 
It does not provide the exact number of bins required for accurately calculating 
channel capacity. Start with a value for n_bins roughly 1/10th of the 'fd' estimates, 
compute the channel capacity, then increase the n_bins and repeat the calculation.
"""
bin_set = [10,20,40,80,160]

In [149]:
of = open('C_summary.csv','w')
print('Bins,C',file=of)
of.close()

try:
    os.chdir(data_directory+'/response')
except OSError:
    os.mkdir(data_directory+'/response')
    os.chdir(data_directory+'/response')

In [150]:
bao = BA()

# Number of bins

for n_bins in bin_set:
    bin_edges = np.linspace(1.0,glob_max + offset,n_bins+1)
    
    # Compute and write probability transition matrix
    pdfs = np.zeros(shape=(len(conclist),n_bins))
    for j in range(0,len(conclist)):
        hist, bin_edges = np.histogram(datas[conc],bins=bin_edges)
        pdfs[j,:] = hist/np.sum(hist)
        
    np.savetxt('expressions.csv',pdfs,delimiter=',')
    
    # Compute probability transition matrix for subsampled data with replicates
    data_fractions = [1,2,5,10]
    n_reps = 5

    c_subsample = np.zeros(shape=(n_reps*len(data_fractions),2))
    idx = 0

    for df in data_fractions:
        for k in range(1,6):
            pdfs = np.zeros(shape=(len(conclist),n_bins))

            for i in range(0,len(conclist)):
                cs = conclist[i]
                darray = datas[cs]
                darray_list = list(darray)
                sample_size = int(len(darray_list)/df)

                d_sampled = rand.sample(darray_list,sample_size)

                hist, b_edges = np.histogram(np.array(d_sampled),bin_edges)

                pdfs[i,:] = hist/np.sum(hist)

            response_file = 'expressions'+str(int(df))+'_'+str(k)+'.csv'
            
            if n_bins!=bin_set[-1]:
                np.savetxt(response_file,pdfs,delimiter=',')
                
            
            # Compute channel capacity
            bao.set_response(pdfs)
            c, e, p = bao.get_CC()

            c_subsample[idx,0] = float(df)
            c_subsample[idx,1] = c
            
            idx += 1

    popt, pcov = curve_fit(linear, c_subsample[:,0], c_subsample[:,1])

    os.chdir('..')
    print(str(int(n_bins))+','+str("{:0.2f}".format(popt[1])),file=open('C_summary.csv','a'))
    os.chdir('response')
    
    print(n_bins,popt[1])

10 0.026688627310025415
20 0.11964272575438128
40 0.47104442105709254
80 0.735148332728254
160 0.773469271085845
